In [5]:
import os
import uuid
import base64
from IPython import display
from unstructured.partition.pdf import partition_pdf
from langchain.chat_models import ChatOpenAI
from langchain.embeddings import OpenAIEmbeddings
from langchain.prompts import PromptTemplate
from langchain.schema.messages import HumanMessage, SystemMessage
from langchain.schema.document import Document
from langchain.retrievers.multi_vector import MultiVectorRetriever

from langchain_core.messages import HumanMessage, SystemMessage

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.vectorstores import Chroma
from langchain.chains import LLMChain
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from openai import OpenAI

from IPython import display

from dotenv import load_dotenv


In [6]:
load_dotenv()

True

In [7]:
os.environ['OPENAI_API_KEY']=os.getenv("OPENAI_API_KEY")

In [8]:
text_model =  ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.5,
    max_tokens=1500,
)

C:\Users\pasin\AppData\Local\Temp\ipykernel_2220\2036851144.py:1: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  text_model =  ChatOpenAI(


In [9]:
def extract_images_and_tables_from_unstructured_pdf(pdf_path,output_path):
    # Load the PDF file
    raw_pdf_elements=partition_pdf(
    filename=pdf_path,                  
    strategy="hi_res",                                
    extract_images_in_pdf=True,                      
    extract_image_block_types=["Image", "Table"],          
    extract_image_block_to_payload=False,                  
    extract_image_block_output_dir=output_path, 
    )
    
    img=[]
    for element in raw_pdf_elements:
        if "unstructured.documents.elements.Image" in str(type(element)):
            img.append(str(element))

    tab=[]
    for element in raw_pdf_elements:
        if "unstructured.documents.elements.Table" in str(type(element)):
            tab.append(str(element))

    
    return img,tab


In [10]:

def encode_image(image_path):
    with open(image_path, "rb") as f:
        return base64.b64encode(f.read()).decode('utf-8')

In [11]:
def summarize_image(encoded_image, vision_model):
    prompt = """You are an assistant tasked with summarizing images for retrieval. \
    These summaries will be embedded and used to retrieve the raw image. \
    Give a concise summary of the image that is well optimized for retrieval. It should be included the most important information in the image. """

    msg = [
        HumanMessage(content=[
            {
                "type": "text",
                "text": prompt
            },
            {
                "type": "image_url",
                "image_url": {
                    "url": f"data:image/jpeg;base64,{encoded_image}"
                },
            },
        ])
    ]
    response = vision_model.invoke(msg)
    return response.content

In [12]:
encoded_image = encode_image("Capture6.PNG")

FileNotFoundError: [Errno 2] No such file or directory: 'Capture6.PNG'

In [13]:
response = summarize_image(encoded_image, text_model)

NameError: name 'encoded_image' is not defined

In [ ]:
response

'Summary: The image outlines the prerequisites for an AWS training course. It details general required knowledge, including IT technical skills, completion of AWS Academy Cloud Foundations, intermediate Python skills, and knowledge of applied statistics. Preferred knowledge includes familiarity with cloud computing concepts, distributed systems, general networking concepts, and multi-tier architectures. Additionally, it emphasizes the importance of general business knowledge, communication skills, and customer service orientation for course success.'

In [45]:
import uuid
def get_summary_of_images(image_elements,output_path, vision_model):
    image_summaries = []
    img_base64_list = []
    img_path_list = []
    for i in os.listdir(output_path):
        if i.endswith(('.png', '.jpg', '.jpeg')):
            image_path = os.path.join(output_path, i)
            encoded_image = encode_image(image_path)
            img_base64_list.append(encoded_image)
            summary = summarize_image(encoded_image, vision_model)
            print("summary", summary)
            image_summaries.append(summary)

        image_path = os.path.join(output_path, i)
        print("rename image path", image_path)
        os.rename(image_path, os.path.join("/images",uuid.uuid4().hex + i))
        #add unique id to image path
        image_path = os.path.join("/images", uuid.uuid4().hex + i)
        
        

        img_path_list.append(image_path)




    print(image_summaries , "image summaries")
    print(img_base64_list , "image base64 list")
    print(img_path_list , "image path")
    return image_summaries,img_base64_list, img_path_list

In [15]:
import uuid
print (uuid.uuid4())

e2669616-43cb-4e4b-8dd6-07cde78151fe


In [58]:
def get_summary_of_tables(table_elements,text_model):
    table_summaries = []

    # Prompt
    prompt_text = """You are an assistant tasked with summarizing tables for retrieval. \
    These summaries will be embedded and used to retrieve the raw table elements. \
    Give a concise summary of the table that is well optimized for retrieval. Table:{element} """
    prompt = ChatPromptTemplate.from_template(prompt_text)

    summarize_chain = {"element": lambda x: x} | prompt | text_model | StrOutputParser()

    table_summaries = summarize_chain.batch(table_elements)

    return table_summaries

In [17]:
#create documents for tables summary
def create_documents_tables(table_elements,table_summaries,documents,retrieve_contents):
    i = str(uuid.uuid4())
    for e, s in zip(table_elements, table_summaries):
        doc = Document(
            page_content = s,
            metadata = {
                'id': i,
                'type': 'table',
                'original_content': e
            }
        )
        retrieve_contents.append((i, e))
        documents.append(doc)

In [49]:
def get_summary_of_images(image_elements,output_path, vision_model):
    image_summaries = []
    img_base64_list = []
    img_path_list = []
    for i in os.listdir(output_path):
        if i.endswith(('.png', '.jpg', '.jpeg')):
            image_path = os.path.join(output_path, i)
            encoded_image = encode_image(image_path)
            img_base64_list.append(encoded_image)
            summary = summarize_image(encoded_image, vision_model)
            image_summaries.append(summary)

        image_path = os.path.join(output_path, i)
        print("rename image path", image_path)

        destination_dir = "./images-processed/"
        os.makedirs(destination_dir, exist_ok=True)
        unique_filename = uuid.uuid4().hex + i
        new_image_path = os.path.join(destination_dir, unique_filename)
        os.rename(image_path, new_image_path)
        #add unique id to image path
        img_path_list.append(new_image_path)

    print(image_summaries , "image summaries")
    # print(img_base64_list , "image base64 list")
    # print(img_path_list , "image path")
    return image_summaries,img_base64_list , img_path_list

In [19]:
def get_documents_text(path_name):
        
    loader = PyPDFLoader(path_name)
    documents = loader.load()
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=150)
    docs = text_splitter.split_documents(documents)
    return docs


In [20]:
def create_documents_text(text_documents,documents):
    i = str(uuid.uuid4())
    for s in text_documents:
        doc = Document(
            page_content = s.page_content,
            metadata = {
                'id': i,
                'type': 'text',
                'original_content': s.metadata['source']
            }
        )
        # print(doc)
        documents.append(doc)


In [21]:

#create vector store
def create_vector_store(documents, embedding_model, dbpath):
    vectorstore = Chroma.from_documents(documents=documents, embedding=embedding_model , persist_directory=dbpath)
    return vectorstore


In [22]:
#create load vector store
def load_vector_store(directory, embedding_model):
    vectorstore = Chroma(
        embedding_function=embedding_model,
        persist_directory=directory
    )
    return vectorstore


In [23]:

#create chain
def create_chain(text_model):
    prompt_template = """
    You are a tutor assistant. You aims to provide personalized instruction, guided problem-solving, and adaptive teaching to cater to each student's unique needs and learning pace.
    Answer the question based only on the following context, which can include text, images and tables:
    {context}
    Question: {question}
    Don't answer if you have no context and decline to answer and say "Sorry, I don't have much information about it."
    Just return the helpful answer in as much as detailed possible.
    Answer:
    """
    qa_chain = LLMChain(llm=text_model,
                        prompt=PromptTemplate.from_template(prompt_template))
    
    return qa_chain

In [24]:
def retrieve_content(query,chain,vectorstore):
    relevant_docs = vectorstore.similarity_search(query)
    print(relevant_docs)
    context = ""
    relevant_images = []
    for d in relevant_docs:
        if d.metadata['type'] == 'text':
            context += '[text]' + d.page_content
        elif d.metadata['type'] == 'table':
            context += '[table]' + d.page_content
        elif d.metadata['type'] == 'image':
            context += '[image]' + d.page_content
            relevant_images.append(d.metadata['original_content'])
    result = chain.run({'context': context, 'question': query})
    print(relevant_images , "relevant images")
    return result, relevant_images

In [40]:
pathname1 = "Python_Operators.pdf"
pathname2 = "What_is_a_Battery.pdf"
text_model = text_model
vision_model = text_model
embedding_model = OpenAIEmbeddings(model="text-embedding-3-small")
fpath="./extracted_data/"
database_path = 'docs/chroma_db/'

In [ ]:
#  call image and table pdf documents
image_elements, table_elements = extract_images_and_tables_from_unstructured_pdf(pathname2,fpath)

In [54]:
#  call image and table pdf documents
image_elements, table_elements = extract_images_and_tables_from_unstructured_pdf(pathname1,fpath)

In [42]:
table_elements

['Learn to code solving problems with our hands-on Python course! Try Programiz PRO today. (hps://programiz.pro/learn/master-python? utm_source=sticky- banner&utm_campaign=programiz&utm_medium=referral) Programiz PRO (hps://programiz.pro/learn/master-python? utm_source=nav- (/) floating&utm_campaign=programiz&utm_medium=referral)',
 'Programiz PRO',
 '** to get a to the power b Learn to code solving problems with our hands-on Python course! Try Programiz PRO today. Programiz PRO (hps://programiz.pro/learn/master-python? utm_source=nav- (/) floating&utm_campaign=programiz&utm_medium=referral) www.domain-name.com',
 'Operator Name Example = Assignment Operator a = 7 += Addition Assignment a += 1 # a = a + 1 -= Subtraction Assignment a -= 3 # a = a - 3 *= Multiplication Assignment a *= 4 # a = a * 4 /= Division Assignment a /= 3 # a = a / 3',
 'Learn to code solving problems with our hands-on Python course! Try Programiz PRO today. a **= 10 # a = a ** 10 Exponent Assignment **= (hps://pro

In [59]:
table_summaries = get_summary_of_tables(table_elements,text_model)

In [60]:
table_summaries

['Summary: Programiz offers a hands-on Python course designed to teach coding through problem-solving. Explore Programiz PRO for more details and enrollment options.',
 '**Table Summary: Programiz PRO Features and Benefits**  \nThis table outlines the key features and benefits of the Programiz PRO subscription, including access to premium content, interactive coding exercises, personalized learning paths, and additional resources for programming languages. It highlights pricing tiers, user testimonials, and comparisons with free offerings, emphasizing the enhanced learning experience for users.',
 '**Table Summary:** This table promotes a hands-on Python coding course offered by Programiz PRO. It includes a link to the course for further details and emphasizes practical problem-solving skills.',
 '**Summary:** This table lists assignment operators in programming, including their names and examples. Key operators include:  \n- `=` (Assignment): `a = 7`  \n- `+=` (Addition Assignment): `

In [55]:
image_summaries, images_list , image_path_list = get_summary_of_images(image_elements,fpath,vision_model)

rename image path ./extracted_data/figure-10-4.jpg
rename image path ./extracted_data/figure-12-5.jpg
rename image path ./extracted_data/figure-12-6.jpg
rename image path ./extracted_data/figure-13-7.jpg
rename image path ./extracted_data/figure-2-1.jpg
rename image path ./extracted_data/figure-6-2.jpg
rename image path ./extracted_data/figure-9-3.jpg
rename image path ./extracted_data/table-1-1.jpg
rename image path ./extracted_data/table-11-10.jpg
rename image path ./extracted_data/table-14-11.jpg
rename image path ./extracted_data/table-3-2.jpg
rename image path ./extracted_data/table-4-3.jpg
rename image path ./extracted_data/table-4-4.jpg
rename image path ./extracted_data/table-5-5.jpg
rename image path ./extracted_data/table-7-6.jpg
rename image path ./extracted_data/table-7-7.jpg
rename image path ./extracted_data/table-8-8.jpg
rename image path ./extracted_data/table-9-9.jpg
['Image summary: A table displaying two logical operators, "in" and "not in," with their meanings and e

In [56]:
image_summaries

['Image summary: A table displaying two logical operators, "in" and "not in," with their meanings and examples. The "in" operator indicates that a value/variable is found in a sequence, while the "not in" operator indicates that a value/variable is not found in the sequence. Examples provided include "5 in x" and "5 not in x."',
 'Image features a webpage promoting a hands-on Python course called "Programiz PRO." It includes links to the course and a "Check Code" button. The page emphasizes coding problem-solving skills and offers multiple URLs for course access.',
 'Image summary: Title "Operators in Python #5" with a prominent YouTube play button, indicating video content related to Python programming operators.',
 'Image summary: The image displays a webpage promoting a Python programming course from Programiz, featuring links to share the article on Facebook and Twitter. It includes a feedback section with smiley face icons for users to indicate if they found the article helpful.',

In [ ]:
image_summaries, images_list  = get_summary_of_images(image_elements,fpath,vision_model)

rename image path ./extracted_data/figure-1-1.jpg
rename image path ./extracted_data/figure-1-2.jpg
rename image path ./extracted_data/figure-1-3.jpg
rename image path ./extracted_data/figure-1-4.jpg
rename image path ./extracted_data/figure-2-5.jpg
rename image path ./extracted_data/figure-2-6.jpg
rename image path ./extracted_data/figure-2-7.jpg
rename image path ./extracted_data/figure-3-8.jpg
rename image path ./extracted_data/figure-3-9.jpg
['Logo of SparkFun, featuring the brand name "sparkfun" in a bold font with the tagline "START SOMETHING" underneath. The logo includes a stylized flame or wrench icon above the text.', 'A person in a suit holds a large, vibrant red heart, symbolizing love or affection, against a plain white background.', 'Image summary: A text-based graphic titled "What is a Battery?" featuring a central YouTube play button icon, indicating a video resource related to the topic of batteries.', 'Image of a lithium polymer battery connected to a USB charging cir

In [26]:
text_documents = get_documents_text(pathname1)

In [27]:
text_documents

[Document(metadata={'source': 'Introduction to_Electric_Motors.pdf', 'page': 0}, page_content='Introduction to Electric Motors\nHi Guys! W elcome you onboard. Interested to know the Introduction to  Electric Motors ?Keep reading…A   motor  is an electrical\nmachine...\nPosted at: 08 - Sep - 2020 Category: Tutorials Author: jameswilson 0 Comments\nHi Guys! W elcome you onboard. Interested to know the Introduction to Electric Motors? Keep\nreading… A motor is an electrical machine that converts electrical energy into mechanical\nenergy . It works exactly opposite to generator that converts mechanical energy to electrical\nenergy . The first electric motors were introduced by Andrew Gordon and Benjamin Franklin in\ntheir experiment in 1740 which were nothing but electrostatic devices. From household to\nindustrial applications, you’ll see motors everywhere. Motors can be divided into two main\ncategories:\nMotors in cars, rectifiers, and batteries are a source of direct current motors whi

In [28]:
from PyPDF2 import PdfReader
from langchain.text_splitter import CharacterTextSplitter

def get_pdf_text(pdf_docs):
    text = ""
    for pdf in pdf_docs:
        pdf_reader = PdfReader(pdf)
        for page in pdf_reader.pages:
            text += page.extract_text()
    return text


def get_text_chunks(text):
    text_splitter = CharacterTextSplitter(
        separator="\n",
        chunk_size=1000,
        chunk_overlap=200,
        length_function=len
    )
    chunks = text_splitter.split_text(text)
    return chunks



In [29]:

pdf_docs = [pathname1]
text = get_pdf_text(pdf_docs)

In [30]:
text

"Introduction to Electric Motors\nHi Guys! W elcome you onboard. Interested to know the Introduction to  Electric Motors ?Keep reading…A   motor  is an electrical\nmachine...\nPosted at: 08 - Sep - 2020 Category: Tutorials Author: jameswilson 0 Comments\nHi Guys! W elcome you onboard. Interested to know the Introduction to Electric Motors? Keep\nreading… A motor is an electrical machine that converts electrical energy into mechanical\nenergy . It works exactly opposite to generator that converts mechanical energy to electrical\nenergy . The first electric motors were introduced by Andrew Gordon and Benjamin Franklin in\ntheir experiment in 1740 which were nothing but electrostatic devices. From household to\nindustrial applications, you’ll see motors everywhere. Motors can be divided into two main\ncategories:\nMotors in cars, rectifiers, and batteries are a source of direct current motors while motors incorporated in electrical generators, power grid\nstations, and invertors operate b

In [32]:
list  = get_text_chunks(text)

In [33]:
len(list)

22

In [34]:
list[0]

'Introduction to Electric Motors\nHi Guys! W elcome you onboard. Interested to know the Introduction to  Electric Motors ?Keep reading…A   motor  is an electrical\nmachine...\nPosted at: 08 - Sep - 2020 Category: Tutorials Author: jameswilson 0 Comments\nHi Guys! W elcome you onboard. Interested to know the Introduction to Electric Motors? Keep\nreading… A motor is an electrical machine that converts electrical energy into mechanical\nenergy . It works exactly opposite to generator that converts mechanical energy to electrical\nenergy . The first electric motors were introduced by Andrew Gordon and Benjamin Franklin in\ntheir experiment in 1740 which were nothing but electrostatic devices. From household to\nindustrial applications, you’ll see motors everywhere. Motors can be divided into two main\ncategories:\nMotors in cars, rectifiers, and batteries are a source of direct current motors while motors incorporated in electrical generators, power grid'

In [35]:
def create_documents_text(text_list ,documents):
    i = str(uuid.uuid4())
    for s in text_list:
        doc = Document(
            page_content = s,
            metadata = {
                'id': i,
                'type': 'text',
        
            }
        )
        # print(doc)
        documents.append(doc)

In [36]:
documents = []

In [37]:
create_documents_text(list,documents)

In [38]:
documents

[Document(metadata={'id': '16cd9916-b62d-42f4-b56e-2484d2401ef3', 'type': 'text'}, page_content='Introduction to Electric Motors\nHi Guys! W elcome you onboard. Interested to know the Introduction to  Electric Motors ?Keep reading…A   motor  is an electrical\nmachine...\nPosted at: 08 - Sep - 2020 Category: Tutorials Author: jameswilson 0 Comments\nHi Guys! W elcome you onboard. Interested to know the Introduction to Electric Motors? Keep\nreading… A motor is an electrical machine that converts electrical energy into mechanical\nenergy . It works exactly opposite to generator that converts mechanical energy to electrical\nenergy . The first electric motors were introduced by Andrew Gordon and Benjamin Franklin in\ntheir experiment in 1740 which were nothing but electrostatic devices. From household to\nindustrial applications, you’ll see motors everywhere. Motors can be divided into two main\ncategories:\nMotors in cars, rectifiers, and batteries are a source of direct current motors w

In [39]:
import streamlit as st

# File uploader for PDFs
uploaded_file = st.file_uploader("Upload a PDF", type=["pdf"])

if uploaded_file is not None:
    # Save the uploaded file locally
    with open(f"uploaded_{uploaded_file.name}", "wb") as f:
        f.write(uploaded_file.read())
    
    st.success(f"File {uploaded_file.name} has been saved successfully as uploaded_{uploaded_file.name}")


2024-09-21 10:35:27.640 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-21 10:35:27.641 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-21 10:35:27.648 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-21 10:35:27.871 
  command:

    streamlit run d:\Documnets\ML\LLM\New-multimodel-pdf-llm\env\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
2024-09-21 10:35:27.872 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
